lstm model


In [ ]:
import os
import re
import pandas as pd

# Define paths to the data files
lines_file_path = 'movie_lines.txt'
conversations_file_path = 'movie_conversations.txt'

dataset_size = 10000  # Adjust this value as needed

# Function to load and process movie lines
def load_movie_lines(file_path):
    id_to_line = {}
    with open(file_path, 'r', encoding='iso-8859-1') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                line_id, text = parts[0], parts[4]
                # Clean text
                text = re.sub(r"([?.!,])", r" \1 ", text)  # Space out punctuation
                text = re.sub(r'\s+', ' ', text).strip()
                id_to_line[line_id] = text
    return id_to_line

# Function to load and process conversations with a limit on the number of samples
def load_conversations(file_path, id_to_line, limit=None):
    conversations = []
    with open(file_path, 'r', encoding='iso-8859-1') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                # Extract conversation line IDs and get corresponding text
                line_ids = eval(parts[3])  # Converts string list to list
                conv = [id_to_line[line_id] for line_id in line_ids if line_id in id_to_line]
                if len(conv) > 1:
                    conversations.append(conv)
                # Stop if limit is reached
                if limit and len(conversations) >= limit:
                    break
    return conversations

# Load movie lines and conversations with a dataset size limit
id_to_line = load_movie_lines(lines_file_path)
conversations = load_conversations(conversations_file_path, id_to_line, limit=dataset_size)

# Example: Display first conversation pair
print("Sample Conversation:", conversations[0][:2])

# Create training pairs: (input, response) for each conversation
train_pairs = []
for conv in conversations:
    for i in range(len(conv) - 1):
        input_text, response_text = conv[i], conv[i + 1]
        train_pairs.append((input_text, response_text))

print("Total training pairs:", len(train_pairs))
print("Sample training pair:", train_pairs[0])

Sample Conversation: ['Can we make this quick ? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad . Again .', "Well , I thought we'd start with pronunciation , if that's okay with you ."]
Total training pairs: 25152
Sample training pair: ('Can we make this quick ? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad . Again .', "Well , I thought we'd start with pronunciation , if that's okay with you .")


In [ ]:
import os
import re
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

# Variable to control the size of the dataset used
dataset_size = 10000 # Adjust this value to use a smaller or larger dataset

# Load DistilGPT-2 and tokenizer
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token to avoid padding issues

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)


# Subset the dataset to the desired size
train_pairs_subset = train_pairs[:dataset_size]

# Prepare dataset by encoding conversation pairs
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, tokenizer, max_length=50):
        self.inputs = []
        for input_text, response_text in pairs:
            # Encode the conversation pair with padding and truncation
            encodings = tokenizer(
                input_text + tokenizer.eos_token + response_text + tokenizer.eos_token,
                truncation=True,
                max_length=max_length,
                padding='max_length',  # Ensures each sequence has max_length
                return_tensors="pt"
            )
            input_ids = encodings["input_ids"].squeeze()
            # Set labels to be the same as input_ids
            self.inputs.append({
                "input_ids": input_ids,
                "labels": input_ids.clone()
            })

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx]

# Create the dataset
train_dataset = ChatDataset(train_pairs_subset, tokenizer)

# Define a data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # Set mlm to False for causal language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    evaluation_strategy="no",  # Set to "steps" or "epoch" if validation data is available
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Enable mixed precision if CUDA is available
)

# Use Trainer API for easy training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator  # Use the data collator for proper padding
)

# Train model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,4.268200
200,4.084200
300,3.927400
400,3.949000
500,3.939000
600,3.908600
700,3.837000
800,3.827700
900,3.821100
1000,3.839200


TrainOutput(global_step=7500, training_loss=3.4194492065429687, metrics={'train_runtime': 580.1089, 'train_samples_per_second': 51.714, 'train_steps_per_second': 12.929, 'total_flos': 382758912000000.0, 'train_loss': 3.4194492065429687, 'epoch': 3.0})

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load DistilGPT-2 and tokenizer
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Initialize pipelines for different NLP tasks
text_generation = pipeline("text-generation", model=model, tokenizer=tokenizer)
text_completion = pipeline("text-generation", model=model, tokenizer=tokenizer)  # Reuse for text completion

# Task 1: Text Generation
print("=== Text Generation ===")
prompt_text = "In a world where artificial intelligence"
generated_texts = text_generation(prompt_text, max_length=50, num_return_sequences=1)
for text in generated_texts:
    print(text["generated_text"])

# Task 2: Text Summarization (approximation, since GPT-2 isn’t fine-tuned for summarization)
print("\n=== Text Summarization (Approximation) ===")
summary_prompt = (
    "Machine learning models are trained using data and can improve automatically through experience. "
    "Recent advancements have led to significant breakthroughs in NLP, allowing models to understand and generate human language."
)
summarization_result = text_generation(summary_prompt, max_length=50, num_return_sequences=1)
for text in summarization_result:
    print(text["generated_text"])

# Task 3: Text Completion
print("\n=== Text Completion ===")
completion_prompt = "The future of AI and machine learning will be shaped by"
completion_result = text_completion(completion_prompt, max_length=50, num_return_sequences=1)
for text in completion_result:
    print(text["generated_text"])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more preci

=== Text Generation ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a world where artificial intelligence can help to change everything, it could become a reality -- and a way to get there.

=== Text Summarization (Approximation) ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Machine learning models are trained using data and can improve automatically through experience. Recent advancements have led to significant breakthroughs in NLP, allowing models to understand and generate human language.













It

=== Text Completion ===
The future of AI and machine learning will be shaped by an interplay between AI and machines. The result could include future efforts by AI scientists, computer scientists and engineers in AI to better understand each other, to strengthen communication between both men and women,


In [ ]:
from transformers import pipeline

# Load DistilGPT-2 and tokenizer for the food-themed tests
text_generation = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Task 1: Food-Themed Text Generation
print("=== Food-Themed Text Generation ===")
prompt_food = "The chef prepared a delicious meal that included"
generated_food_texts = text_generation(prompt_food, max_length=50, num_return_sequences=1)
for text in generated_food_texts:
    print(text["generated_text"])

# Task 2: Recipe Suggestion (approximated by the text generation task)
print("\n=== Recipe Suggestion ===")
recipe_prompt = "For a quick pasta dish, start with olive oil, garlic, and add"
recipe_suggestion = text_generation(recipe_prompt, max_length=60, num_return_sequences=1)
for text in recipe_suggestion:
    print(text["generated_text"])

# Task 3: Food Description
print("\n=== Food Description ===")
description_prompt = "Describe the flavor of a freshly baked chocolate cake."
food_description = text_generation(description_prompt, max_length=50, num_return_sequences=1)
for text in food_description:
    print(text["generated_text"])

# Task 4: Food Q&A
print("\n=== Food Q&A ===")
qa_prompt_food = "Question: What are the best spices for a curry? Answer:"
qa_result_food = text_generation(qa_prompt_food, max_length=50, num_return_sequences=1)
for text in qa_result_food:
    print(text["generated_text"])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Food-Themed Text Generation ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The chef prepared a delicious meal that included mashed potatoes and a salad with salt. It was great as an attempt at a special way to make it a dessert.


Ingredients
1 bag of mushrooms
1 tbsp butter
1 large onion,

=== Recipe Suggestion ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


For a quick pasta dish, start with olive oil, garlic, and add more oregano and onion to get a pretty good start. Just use the olive oil and pepper and mix with the onion in the pan and add another 1/2 of the oregano. Pour the pasta into a

=== Food Description ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Describe the flavor of a freshly baked chocolate cake.






In the next paragraph we'll discuss an experiment in using peanut butter for making the chocolate cake.
First let me know what you think about this tutorial:


=== Food Q&A ===
Question: What are the best spices for a curry? Answer: I enjoy it because I love it because i like it because it reminds me of curry is a great spice for a lot of the people in my house that I know. I like to


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForCausalLM
import torch

def fine_tune_model(model_name, train_pairs_subset, max_length=50, batch_size=4, num_epochs=3):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token if hasattr(tokenizer, 'eos_token') else tokenizer.pad_token

    model = AutoModelForCausalLM.from_pretrained(model_name)  # Adjust for the correct model class
    model.to(device)

    # Prepare the dataset
    train_dataset = ChatDataset(train_pairs_subset, tokenizer, max_length=max_length)

    # Define a data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
        evaluation_strategy="no",
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),
    )

    # Use Trainer API for easy training
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator
    )

    # Train model
    try:
        trainer.train()
    except Exception as e:
        print(f"Error during training: {e}")

    # Ensure all tensors are contiguous after training
    for param in model.parameters():
        param.data = param.data.contiguous()


In [ ]:
pip install --upgrade transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define your dataset size and prepare the pairs
dataset_size =  10000 # Adjust as needed
train_pairs_subset = train_pairs[:dataset_size]

# Fine-tune XLNet
fine_tune_model("xlnet-base-cased", train_pairs_subset)

# Fine-tune ALBERT
fine_tune_model("roberta-base", train_pairs_subset)




NameError: name 'train_pairs' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration
import torch

# Ensure you have a device set up (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dataset class
class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.labels[idx],
        }

def fine_tune_flan_t5(train_pairs_subset, model_name="google/flan-t5-small", max_length=50, batch_size=4, num_epochs=3):
    # Load the tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model.to(device)

    # Prepare the dataset
    inputs = tokenizer([f"Translate English to French: {pair[0]}" for pair in train_pairs_subset],
                       padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    labels = tokenizer([pair[1] for pair in train_pairs_subset],
                       padding=True, truncation=True, max_length=max_length, return_tensors="pt")["input_ids"]

    # Create the dataset
    dataset = T5Dataset(inputs, labels)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
        save_steps=500,
        save_total_limit=2,
        logging_steps=100,
        evaluation_strategy="no",
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),
    )

    # Use Trainer API for easy training
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    # Train model
    try:
        trainer.train()
    except Exception as e:
        print(f"Error during training: {e}")

# Example usage with dummy data
train_pairs_subset = [
    ("Hello, how are you?", "I'm fine, thank you!"),
    ("What is your name?", "My name is ChatGPT."),
    # Add more conversation pairs as needed
]

fine_tune_flan_t5(train_pairs_subset)


In [ ]:
!pip install --upgrade datasets


In [ ]:
!pip install evaluate


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(train_losses, perplexities, bleu_scores):
    epochs = range(1, len(train_losses) + 1)

    # Plot Loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_losses, label="Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.legend()

    # Plot Perplexity
    plt.subplot(1, 3, 2)
    plt.plot(epochs, perplexities, label="Perplexity", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("Perplexity")
    plt.title("Perplexity over Time")
    plt.legend()

    # Plot BLEU Score
    plt.subplot(1, 3, 3)
    plt.plot(epochs, bleu_scores, label="BLEU Score", color="green")
    plt.xlabel("Epochs")
    plt.ylabel("BLEU Score")
    plt.title("BLEU Score over Time")
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage with dummy data
train_losses = [2.3, 1.9, 1.5]  # Replace with actual loss values per epoch
perplexities = [20, 15, 12]     # Replace with actual perplexity values per epoch
bleu_scores = [0.25, 0.30, 0.35]  # Replace with actual BLEU scores per epoch

plot_metrics(train_losses, perplexities, bleu_scores)


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Define model paths
model_paths = {
    "xlnet": "xlnet-base-cased",
    "roberta": "roberta-base",
    "flan-t5": "google/flan-t5-small"
}

# Load models and tokenizers
models = {}
tokenizers = {}
for name, path in model_paths.items():
    if name == "flan-t5":
        models[name] = T5ForConditionalGeneration.from_pretrained(path).eval()
    else:
        models[name] = AutoModelForSequenceClassification.from_pretrained(path).eval()
    tokenizers[name] = AutoTokenizer.from_pretrained(path)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model in models.values():
    model.to(device)

# Evaluation function for confidence and pseudo-perplexity
def evaluate_models(models, tokenizers, sample_text):
    results = {}

    for name, model in models.items():
        tokenizer = tokenizers[name]
        inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True).to(device)

        # Model-specific evaluation
        with torch.no_grad():
            if name != "flan-t5":
                # For Sequence Classification (XLNet, RoBERTa)
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = F.softmax(logits, dim=-1)
                confidence = torch.max(probabilities, dim=-1).values.cpu().item()
                results[name] = {"confidence_score": confidence}
            else:
                # For T5 (calculate pseudo-perplexity based on decoder input requirement)
                # Shift input ids by one for decoder inputs
                decoder_input_ids = inputs['input_ids'].clone()
                outputs = model(input_ids=inputs['input_ids'], decoder_input_ids=decoder_input_ids)
                logits = outputs.logits
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
                    decoder_input_ids.view(-1),
                    ignore_index=tokenizer.pad_token_id,
                    reduction="mean"
                )
                perplexity = torch.exp(loss).cpu().item()
                results[name] = {"pseudo_perplexity": perplexity}

    return results

# Test with a sample input
sample_text = "What are the benefits of a healthy diet?"
results = evaluate_models(models, tokenizers, sample_text)
print("Evaluation Results:", results)


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Define model paths
model_paths = {
    "xlnet": "xlnet-base-cased",
    "roberta": "roberta-base",
    "flan-t5": "google/flan-t5-small"
}

# Load models and tokenizers
models = {}
tokenizers = {}
for name, path in model_paths.items():
    if name == "flan-t5":
        models[name] = T5ForConditionalGeneration.from_pretrained(path).eval()
    else:
        models[name] = AutoModelForSequenceClassification.from_pretrained(path).eval()
    tokenizers[name] = AutoTokenizer.from_pretrained(path)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model in models.values():
    model.to(device)

# Evaluation function for individual models
def evaluate_model(model, tokenizer, text, model_name):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        if model_name != "flan-t5":
            # For Sequence Classification (XLNet, RoBERTa)
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = F.softmax(logits, dim=-1)
            confidence = torch.max(probabilities, dim=-1).values.cpu().item()
            return confidence
        else:
            # For T5 (calculate pseudo-perplexity based on decoder input requirement)
            decoder_input_ids = inputs['input_ids'].clone()
            outputs = model(input_ids=inputs['input_ids'], decoder_input_ids=decoder_input_ids)
            logits = outputs.logits
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                decoder_input_ids.view(-1),
                ignore_index=tokenizer.pad_token_id,
                reduction="mean"
            )
            perplexity = torch.exp(loss).cpu().item()
            return perplexity

# Ensemble evaluation function
def ensemble_evaluate(models, tokenizers, text):
    results = {}
    total_confidence = 0
    total_perplexity = 0
    count = 0

    for name, model in models.items():
        score = evaluate_model(model, tokenizers[name], text, name)

        if name != "flan-t5":
            results[name] = {"confidence_score": score}
            total_confidence += score
        else:
            results[name] = {"pseudo_perplexity": score}
            total_perplexity += score
        count += 1

    # Calculate the average metrics for ensemble performance
    avg_confidence = total_confidence / (count - 1)  # -1 as T5 gives perplexity
    avg_perplexity = total_perplexity / 1  # Only one perplexity score (from T5)

    # Final combined results
    results["ensemble"] = {
        "average_confidence_score": avg_confidence,
        "average_pseudo_perplexity": avg_perplexity
    }
    return results

# Sample input
sample_text = "What are the benefits of a healthy diet?"
results = ensemble_evaluate(models, tokenizers, sample_text)
print("Ensemble Evaluation Results:", results)


In [ ]:
import os
import re
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)


In [ ]:
# Define paths to the data files
lines_file_path = 'movie_lines.txt'
conversations_file_path = 'movie_conversations.txt'

# Dynamic dataset size parameter
dataset_size = 50  # Adjust this value for testing


In [ ]:
# Function to load and process movie lines
def load_movie_lines(file_path):
    id_to_line = {}
    with open(file_path, 'r', encoding='iso-8859-1') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                line_id, text = parts[0], parts[4]
                # Clean text
                text = re.sub(r"([?.!,])", r" \1 ", text)  # Space out punctuation
                text = re.sub(r'\s+', ' ', text).strip()
                id_to_line[line_id] = text
    return id_to_line

# Function to load and process conversations with a limit on the number of samples
def load_conversations(file_path, id_to_line, limit=None):
    conversations = []
    with open(file_path, 'r', encoding='iso-8859-1') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                line_ids = eval(parts[3])  # Converts string list to list
                conv = [id_to_line[line_id] for line_id in line_ids if line_id in id_to_line]
                if len(conv) > 1:
                    conversations.append(conv)
                if limit and len(conversations) >= limit:
                    break
    return conversations


In [ ]:
# Load movie lines and conversations with a dataset size limit
id_to_line = load_movie_lines(lines_file_path)
conversations = load_conversations(conversations_file_path, id_to_line, limit=dataset_size)

# Create training pairs: (input, response) for each conversation
train_pairs = []
for conv in conversations:
    for i in range(len(conv) - 1):
        input_text, response_text = conv[i], conv[i + 1]
        train_pairs.append((input_text, response_text))

print("Total training pairs:", len(train_pairs))


In [ ]:
model_names = {
    "xlnet": "xlnet-base-cased",
    "roberta": "roberta-base",
    "flant5": "google/flan-t5-small"
}

tokenizers = {}
models = {}

for name, model_path in model_names.items():
    tokenizers[name] = AutoTokenizer.from_pretrained(model_path)
    # Use AutoModelForSeq2SeqLM for "flant5"
    if name == "flant5":
        from transformers import AutoModelForSeq2SeqLM # Import the correct model class
        models[name] = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    else:
        models[name] = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
# Prepare dataset for training
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.pairs = pairs
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        input_text, response_text = self.pairs[idx]
        inputs = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        labels = self.tokenizer(response_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = labels['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


In [ ]:
# Training function for each model
def train_model(model, tokenizer, train_data):
    dataset = ChatDataset(train_data, tokenizer)

    training_args = TrainingArguments(
        output_dir='./results',          # Output directory for model predictions and checkpoints
        per_device_train_batch_size=4,   # Batch size for training
        num_train_epochs=3,              # Total number of training epochs
        logging_dir='./logs',            # Directory for storing logs
        logging_steps=10,                 # Log every 10 steps
        save_steps=500,                   # Save model every 500 steps
        save_total_limit=2,               # Limit the total amount of checkpoints
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    trainer.train()


In [ ]:
# Train each model
for name, model in models.items():
    print(f"Training {name} model...")
    train_model(model, tokenizers[name], train_pairs)

print("Training completed!")


In [ ]:
def generate_response(model, tokenizer, input_text, max_length=50):
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate output from the model
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

    # Decode the generated output
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response


In [ ]:
def evaluate_models(models, tokenizers, prompts):
    results = {}
    for name in models.keys():
        responses = []
        for prompt in prompts:
            response = generate_response(models[name], tokenizers[name], prompt)
            responses.append(response)
        results[name] = responses
    return results


In [ ]:
prompts = [
    "What are the benefits of exercise?",
    "Tell me a story about a brave knight.",
    "What is the future of AI?",
    "Explain the importance of a healthy diet."
]

# Evaluate models on the given prompts
evaluation_results = evaluate_models(models, tokenizers, prompts)

# Print the results
for model_name, responses in evaluation_results.items():
    print(f"Responses from {model_name}:")
    for prompt, response in zip(prompts, responses):
        print(f"Prompt: {prompt}")
        print(f"Response: {response}\n")
